In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim
import torch.nn.utils.rnn as rnn

In [3]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [15]:
usr_sys = []
y = []
utt_list = []
errors = ["Ignore question", "Ignore offer", "Ignore proposal", "Ignore greeting"]
for conv in convs:
    for i, ut in enumerate( conv ) :
        utt_list.append(ut.utt)
        # システム発話で，無視系統のエラー
        if ut.is_system() and ut.is_exist_error() and not ut.is_utt_level_error():
        # if ut.is_system():
            usr_sys.append( [conv[i-1].utt, ut.utt] )
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [16]:
vocab_path = "../X_y_data/base/"
vocab_name = "vocab_dict.pickle"
vocabM = DataManager(vocab_path)
vocab_dict = vocabM.load_data(vocab_name)

success load : ../X_y_data/base/vocab_dict.pickle


In [17]:
def sentence2ids(sentence:str, vocab_dict:dict):
    doc = nlp(sentence)
    ids = np.zeros(len(doc))
    for i, token in enumerate(doc):
        key = token.orth_
        if key in vocab_dict:
            ids[i] = vocab_dict[key]
        else:
            ids[i] = vocab_dict["[UNK]"]
    return ids


In [18]:
from tqdm import tqdm
import torch.nn.utils.rnn as rnn

def padding_vector(Xseq):
    Xseq = [ torch.tensor( xseq[:, None] ) for xseq in Xseq]
    Xseq = rnn.pad_sequence(Xseq, batch_first=True)
    Xseq = [ torch.flatten(xseq) for xseq in Xseq ] 
    return Xseq


def make_X(usr_sys:list, vocab_dict:dict):
    usr_id_list = []
    sys_id_list = []
    for turn in tqdm( usr_sys ) :
        usr_id = sentence2ids(turn[0], vocab_dict)
        usr_id_list.append(usr_id)

        sys_id = sentence2ids(turn[1], vocab_dict)
        sys_id_list.append(sys_id)
    
    # usr_id_pad = rnn.pad_sequence(torch.Tensor( usr_id_list) , batch_first=True)
    # sys_id_pad = rnn.pad_sequence(torch.Tensor( sys_id_list), batch_first=True)
    usr_id_pad = padding_vector(usr_id_list)
    sys_id_pad = padding_vector(sys_id_list)

    usr_pad_len = len(usr_id_pad[0])
    sys_pad_len = len(sys_id_pad[0])
    # X =   [ torch.Tensor([u, s]) for u, s in zip(usr_id_pad, sys_id_pad) ] 
    # print(usr_pad_len, sys_pad_len)
    X = torch.zeros( (len(usr_sys), usr_pad_len+sys_pad_len) )
    for i, (u, s) in enumerate( zip(usr_id_pad, sys_id_pad) ):
        # print(i, u, s)
        X[i, :usr_pad_len] = u
        X[i, usr_pad_len: usr_pad_len+sys_pad_len] = s
    return X, usr_pad_len, sys_pad_len


In [19]:
X, upl, spl = make_X(usr_sys, vocab_dict)

100%|██████████| 976/976 [00:20<00:00, 47.09it/s]


In [20]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size, batch_size):
        # 親クラスのコンストラクタ。決まり文句
        super(LSTMClassifier, self).__init__()
        # 隠れ層の次元数。これは好きな値に設定しても行列計算の過程で出力には出てこないので。    
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim,  padding_idx=0)
        # モデルを2つ定義
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True,  bidirectional=True )
        self.lstm2 = nn.LSTM(embedding_dim, hidden_dim//2, batch_first=True,  bidirectional=True )
        self.hidden2tag = nn.Linear(hidden_dim , tagset_size)
        # softmaxのLog版。dim=0で列、dim=1で行方向を確率変換。
        self.softmax = nn.LogSoftmax()
    
    def forward(self, x, upl, spl):
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)

        # x : [seq]
        usr_ = x[:, :upl]
        sys_ = x[:, upl:upl+spl]
        emb1 = self.word_embeddings(usr_)
        emb2 = self.word_embeddings(sys_)
        _, lstm1_out = self.lstm1(emb1)
        _, lstm2_out = self.lstm1(emb2)
        # print(hidden_layer)
        # bilstm_out = torch.cat([lstm_out[0][0], lstm_out[0][1]], dim=1)
        
        usr_vec = ( lstm1_out[0][0] + lstm1_out[0][1] )/2 
        sys_vec = ( lstm2_out[0][0] + lstm2_out[0][1] )/2

        # print(usr_vec.shape, sys_vec.shape)
        # print(torch.cat([ usr_vec, sys_vec], dim=1).shape)
        tag_space = self.hidden2tag(torch.cat([ usr_vec, sys_vec], dim=1 ))
        
        # y = self.hidden2tag(hidden_layer[0].view(batch_size, -1))

        # y = self.hidden2tag(bilstm_out)
        y =self.softmax(tag_space)
        return y

In [21]:
model_path = "../models/base/"
model_name = "responce_form.pickle"
modelM = DataManager(model_path)

In [22]:
model = modelM.load_data(model_name)

success load : ../models/base/responce_form.pickle


In [23]:
with torch.no_grad():
    X_tensor = torch.tensor(X, device='cuda:0', dtype=torch.int)
    y_tensor = torch.tensor(y, device='cuda:0', dtype=torch.long)
            # 推論
    y_pred= np.array(model(X_tensor, upl, spl).cpu()).argmax(axis=1)

/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/yamada/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:662: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:915.)
  self.dropout, self.training, self.bidirectional, self.batch_first)
/home/yamada/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [24]:
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[582  40]
 [ 74 280]]
accuracy =  0.8831967213114754
precision =  0.875
recall =  0.7909604519774012
f1 score =  0.830860534124629


- ベースラインつっよ！(全データ)

        confusion matrix = 
        [[955  77]
        [ 74 280]]
        accuracy =  0.8910533910533911
        precision =  0.7843137254901961
        recall =  0.7909604519774012
        f1 score =  0.7876230661040786

- 発話レベルエラーを除く理想値

        confusion matrix = 
        [[582  40]
        [ 74 280]]
        accuracy =  0.8831967213114754
        precision =  0.875
        recall =  0.7909604519774012
        f1 score =  0.830860534124629